In [1]:
import pandas as pd 
import numpy as np 
import sys
import dask.bag as db

import json
import dask.bag as db
import os

sys.path.append("..")

from pathlib import Path

import json

NOTE: The input data must be downloaded first from https://www.kaggle.com/datasets/Cornell-University/arxiv

In [2]:
ai_category_list=['q-fin']
records=db.read_text("../input/arxiv/arxiv-metadata-oai-snapshot.json").map(lambda x:json.loads(x))
ai_docs = (records.filter(lambda x:any(ele in x['categories'] for ele in ai_category_list)==True))
get_metadata = lambda x: {'id': x['id'],
                  'title': x['title'],
                  'category':x['categories'],
                  'abstract':x['abstract'],
                 'version':x['versions'][-1]['created'],
                         'doi':x["doi"],
                         'authors_parsed':x['authors_parsed']}

data=ai_docs.map(get_metadata).to_dataframe().compute()

In [3]:
data.shape

(15741, 7)

In [11]:
search_original = "cryptocurrency|bitcoin|ethereum"
cryptocurrency = data.loc[data['title'].str.contains(search_original, case=False)]
original = list(cryptocurrency['id'])

In [18]:
search_expanded = "cryptocurrency|bitcoin|ethereum|Ether|solana|Avalanche|Tether \
|BNB|Binance|Dodecoin|Polygon|XRP|Cordano|USD Coin|uniswap|FTX|digital currency| \
digital asset"
cryptocurrency = data.loc[data['title'].str.contains(search_expanded, case=False)]
expanded = list(cryptocurrency['id'])

In [19]:
print(len(expanded))
print(len(original))
new = list(filter(lambda x: x not in original, expanded))
new

245
209


['0705.4329',
 '0907.3273',
 '1010.4990',
 '1104.0777',
 '1205.1617',
 '1307.2181',
 '1309.3639',
 '1506.05911',
 '1611.05688',
 '1701.00993',
 '1805.12101',
 '1810.06696',
 '1911.03380',
 '2005.12949',
 '2005.14631',
 '2006.03023',
 '2010.01241',
 '2010.07404',
 '2010.15611',
 '2011.05839',
 '2101.00281',
 '2102.03172',
 '2102.11807',
 '2103.00254',
 '2104.10973',
 '2106.10012',
 '2106.14404',
 '2109.02452',
 '2110.09429',
 '2111.09192',
 '2112.09783',
 '2203.07774',
 '2205.08904',
 '2208.09642',
 'physics/0507101',
 'physics/0601171']

In [17]:
count = {}
for term in search_expanded.split('|'):
    count[term] = len(data[data['title'].str.contains(term, case=False)])
count   

{'cryptocurrency': 87,
 'bitcoin': 122,
 'ethereum': 5,
 'Ether': 19,
 'solana': 0,
 'Avalanche': 5,
 'Tether ': 1,
 'BNB': 4,
 'Binance': 1,
 'Dodecoin': 0,
 'Polygon': 0,
 'XRP': 1,
 'Cordano': 0,
 'USD Coin': 0,
 'uniswap': 6,
 'FTX': 0,
 'digital currency': 5,
 'digital asset': 5}